In [ ]:
%pip install youtube-transcript-api==0.5.0  # YouTube lib to get transcript
%pip install pytube==12.1.2                 # YouTube lib
%pip install transformers==4.26.1           # Used to know the token size
%pip install tiktoken==0.3.3
%pip install openai==0.27.2
%pip install langchain==0.0.134
%pip install python-dotenv==1.0.0

In [2]:
import os
from dotenv import dotenv_values

api_keys = dotenv_values('keys.txt')
os.environ['OPENAI_API_KEY'] = api_keys['OPENAI_API_KEY']

In [6]:
from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader("FtyKgybYkaE", add_video_info=True)
result = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [7]:
docs = text_splitter.split_documents(result)
docs

[Document(page_content="[Music] thank you good morning devoxx finally it's my slider I'm good after three years this pandemic stuff and so I'm so happy to be back so welcome welcome uh yeah I just want to take this in give me a second so you've probably noticed hopefully the movie Will Not Crash tomorrow evening but uh of course we have Top Gun the Maverick uh tomorrow evening at uh at in this room at eight o'clock so make sure you're here we're gonna have free popcorn for everybody so that should be a good one it was actually a bit difficult because um yeah we've moved it to beginning of October for obvious reasons and to be clear it was for the sunshine right um but um yeah all the big blockbuster movies actually get out in December so I had to actually really dig and see which movie it would be and we were able from Sony to still show this one which isn't actually displayed anymore in the cinemas who hasn't seen the movie yet okay so I think there's gonna be enough people I've I've 

In [8]:
len(docs)

14

In [3]:
from langchain.llms import OpenAI

llm = OpenAI()

In [9]:
#
# STUFF-ing is the simplest method, whereby you simply stuff all the related data 
# into the prompt as context to pass to the language model. 
# 

# Will FAIL because text is too big
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="stuff", verbose=True)   # <<<<< WILL FAIL!! 
chain.run(result)   



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"[Music] thank you good morning devoxx finally it's my slider I'm good after three years this pandemic stuff and so I'm so happy to be back so welcome welcome uh yeah I just want to take this in give me a second so you've probably noticed hopefully the movie Will Not Crash tomorrow evening but uh of course we have Top Gun the Maverick uh tomorrow evening at uh at in this room at eight o'clock so make sure you're here we're gonna have free popcorn for everybody so that should be a good one it was actually a bit difficult because um yeah we've moved it to beginning of October for obvious reasons and to be clear it was for the sunshine right um but um yeah all the big blockbuster movies actually get out in December so I had to actually really dig and see which movie it would be and we were able from Sony to still show this one which isn't ac

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 6009 tokens (5753 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [10]:
#
# MAP REDUCE : involves an initial prompt on each chunk of data 
# 
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(docs)



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"[Music] thank you good morning devoxx finally it's my slider I'm good after three years this pandemic stuff and so I'm so happy to be back so welcome welcome uh yeah I just want to take this in give me a second so you've probably noticed hopefully the movie Will Not Crash tomorrow evening but uh of course we have Top Gun the Maverick uh tomorrow evening at uh at in this room at eight o'clock so make sure you're here we're gonna have free popcorn for everybody so that should be a good one it was actually a bit difficult because um yeah we've moved it to beginning of October for obvious reasons and to be clear it was for the sunshine right um but um yeah all the big blockbuster movies actually get out in December so I had to actually really dig and see which movie it would be and we were able from Sony to still show this one which isn't actually displayed anymore in the

' This speaker is returning to Devoxx after three years and is focusing on the positive use of technology. After two years of virtual events due to COVID-19, they decided to return to physical events. The event was popular and sold out in two seconds. They discussed Project Bloom, Project Amber, Transformers, GitHub Co-Pilot, Dali, e-stable diffusion, and disco diffusion. They also mentioned Devox Ukraine and Beer Central, as well as the YouTube channel they have with 134,000 subscribers. They thanked the speakers, sponsors, and companies like DiffBlue. Finally, they announced Vox Days Brussels in May 2023 and the 20th Edition of a movie trailer project in October.'